In [3]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [4]:
url = "https://archive.ics.uci.edu/dataset/45/heart+disease"

In [8]:
name <- read_table(url)


── Column specification ────────────────────────────────────────────────────────
cols(
  `<!DOCTYPE` = col_character(),
  `html>` = col_character()
)

Warning message:
“310 parsing failures.
row col  expected    actual                                                   file
  1  -- 2 columns 3 columns 'https://archive.ics.uci.edu/dataset/45/heart+disease'
  2  -- 2 columns 1 columns 'https://archive.ics.uci.edu/dataset/45/heart+disease'
  3  -- 2 columns 3 columns 'https://archive.ics.uci.edu/dataset/45/heart+disease'
  4  -- 2 columns 5 columns 'https://archive.ics.uci.edu/dataset/45/heart+disease'
  5  -- 2 columns 8 columns 'https://archive.ics.uci.edu/dataset/45/heart+disease'
... ... ......... ......... ......................................................
See problems(...) for more details.
”


In [10]:
name

<!DOCTYPE,html>
<chr>,<chr>
<html,"lang=""en"""
<head>,NA
<meta,"charset=""UTF-8"""
<meta,"name=""viewport"""
<meta,"name=""description"""
<link,"rel=""icon"""
<link,"rel=""apple-touch-icon"""
<!--,Provides
<!--,See
